In [2]:
import numpy as np
Scal = 128
Count1=0
Count2=0
nbfails=32
L(x)=exp(sqrt(log(x)*log(log(x))))
global Num, Flen, Srange, Startq, CurRow, Count, Bound, Mat_F2
global Mat, Fbas, Rvec, Lvec, Sieve, Pivot, Uvec, Vvec, StackUV

In [3]:
def trial_factor(n):
    factors = []
    d = 2  # Start with the smallest prime factor

    while d * d <= n:
        if n % d == 0:
            factors.append(d)
            n //= d
        else:
            d += 1

    if n > 1:
        factors.append(n)

    return factors

In [4]:
def test_trial_factor():
    n = 274177 * 67280421310721
    print("n = ", n, "=", 274177, "*", 67280421310721)
    factors = trial_factor(n)
    print("factors = ", factors)

In [5]:
test_trial_factor()

n =  18446744073709551617 = 274177 * 67280421310721
factors =  [274177, 67280421310721]


In [6]:
def fermat_factor(N):
    """
    Benutzt das Fermat'sches Faktorisierungs-Verfahren um positive Integer zu faktorisieren

    Parameter:
    - N: Der zu faktorisierende Integerwert

    Rückgabe:
    - ein Tupel (p, q) der Faktoren von N oder None bei negativen/ nicht Integer Werten
    """
    # Prüft den Faktoren 2 und 5
    if is_even(N):
        return (2, N/2)
    if mod(N,5)==0:
        return (5,N/5)

    
    # Try to find a nontrivial square root of n
    x0 = ceil(sqrt(N))
    y_square = x0**2 - N

    while not is_square(y_square):
        x0 += 1
        y_square = x0**2 - N
        if x0>=N :
            return (1,N)
    
    # Berechnet die Zerlegung in (x-y)(x+y)
    p = x0 - sqrt(y_square)
    q = x0 + sqrt(y_square)
    return (p, q)


In [7]:
def test_fermat_factor():
    n = 274177 * 274179
    print("n = ", n, "=", 274177, "*", 274179)
    factors = fermat_factor(n)
    print("factors = ", factors)

In [8]:
test_fermat_factor()

n =  75173575683 = 274177 * 274179
factors =  (274177, 274179)


In [9]:
def make_factorbase(N, B):
    """
    Konstruiert eine Faktorbasis zur gegebenen Zahl N und Länge len

    Parameter:
    - N: Der zu faktorisierende Integerwert
    - len: Die gefordete Länger der Faktorbasis

    Rückgabe:
    - Eine Liste der Länge len von Primzahlen, sodass qudratische Reste mit N entstehen 
    """
    fb = [-1,2]
    p = 2
    while p < B:
        p = next_prime(p)
        while legendre_symbol(N, p) != 1:
            p = next_prime(p)
        fb.append(p)    
    return fb


In [10]:
def test_make_factorbase() :
    N = 157869373
    fb = make_factorbase(N, sqrt(L(N)))
    print("fb = ", fb)

In [11]:
test_make_factorbase()

fb =  [-1, 2, 3, 7, 13, 19, 23, 37, 41, 59]


In [12]:
def rtpvec(N, pvec):
    """
    Berechnet die Wurzel von N im p-Köper der p aus der Faktorbasis 

    Parameter:
    - N: Integer dessen Wurzel berechnet wird
    - pvec: Liste der Zahlen, die die Faktorbasis darstellen

    Rückgabe:
    - Eine Liste der Wurzel von N auf Basis der Faktorbasis
    """
    rvec = [int(mod(N,p).sqrt()) for p in pvec]
    rvec[0]=rvec[1]=1
    return rvec


In [13]:
def test_rtpvec():
    N = 157869373
    fb = make_factorbase(N, 10)
    rvec = rtpvec(N, fb)
    print("rvec = ", rvec)

In [14]:
test_rtpvec()

rvec =  [1, 1, 1, 2, 5]


In [15]:

def logpvec(pvec):
    """
    Berechnet den Logarithmus der p aus der Faktorbasis basierend auf der Skalierung scal

    Parameter:
    - pvec: Liste der Zahlen, die die Faktorbasis darstellen

    Rückgabe:
    - Eine Liste der Logarithmen von N auf Basis der Faktorbasis als Ganzzahlige Werte
    """
    global Scal

    lvec = [0] * len(pvec)
    
    for k in range(1, len(pvec)):
        lvec[k] = round(log(pvec[k]) * Scal)
    
    return lvec

In [16]:
def test_logpvec():
    N = 157869373
    fb = make_factorbase(N, sqrt(L(N)))
    lvec = logpvec(fb)
    print("lvec = ", lvec)

In [17]:
test_logpvec()

lvec =  [0, 89, 141, 249, 328, 377, 401, 462, 475, 522]


In [18]:
def QSinitialize(N):
    """
    Die Funktion QSfactorize(N) initialisiert zuerst die erforderlichen Variablen:

    Parameter:
    - N: Die zu faktorisierende Zahl N.

    Rückgabe:
    - Flen: Die Länge der verwendeten Faktorbasis.
    """

    global Num, Flen, Srange, Startq, CurRow, Count
    global Mat_F2, Mat, Fbas, Rvec, Lvec, Sieve, Pivot, Uvec, Vvec, StackUV
    
    CurRow = 0
    Num = N
    Bound = floor(sqrt(L(Num)))
    Fbas = make_factorbase(Num, Bound)
    Flen = len(Fbas)
    Srange = Bound^2
    Startq = ceil(sqrt(Num)-Srange/2)
    Rvec = rtpvec(N, Fbas)
    Lvec = logpvec(Fbas)
    Uvec =[]
    Vvec =[]
    Mat = MatrixSpace(GF(2),Srange,Flen,sparse=True).matrix()
    Sieve = list(range(Startq,Startq+(Srange)))
    StackUV = []
    
    return Flen


In [19]:
def QSfactorize(N):
    """
    Die Funktion QSfactorize(N) implementiert den Faktorisierungsprozess des Quadratischen Siebs (Quadratic Sieve) für eine gegebene Zahl N.

    Parameter:
    - N: Die zu faktorisierende Zahl N.

    Rückgabe:
    - none
    """    
    
    global Srange, Fbas, CurRow, Uvec, Vvec, Flen, Count1, Count2, nbfails, Mat

    QSinitialize(N)
    print("quadratic sieve length", Srange,
          ", factorbase 2 ...", Fbas[Flen - 1], "of length", Flen - 1)
    print('working ', end='')
    nbfail = 0
    while nbfail < nbfails:
        (u, v) = get_qres(N)
        
        if not qr_trialdiv(v):
            continue
            
        Uvec.append(u)
        Vvec.append(v)
        CurRow += 1
        #print()
        #print(Mat[0:CurRow,:])
        #print(len(Uvec), end='')
        #print(len(Vvec), end='')

        if CurRow % 10 == 0:         # check if matrix is big enough
            #print('\n10erMarke')   
            print('.', end='')   
            Mat_kernel = kernel(Mat[0:CurRow,:])
        else:
            print('.', end='')
            continue
        
        
        if  Mat_kernel.dimension() == 0:          # check if matrix is full rank
            print('.', end='')
        else:
            print('!', end='')
            d = findfactor(N, Mat_kernel)
            if d > 0:
                print()
                print(Count1, "polynomials,", Count2,
                      "completely factorized quadratic residues")
                return d
            else:
                nbfail += 1
    print('\nfailed to often, aborting.... \n')

In [20]:
def get_qres(N):
    """
    Die Funktion get_qres(N) hat den Zweck, Paare von u und v zu generieren, für die u^2 ≡ v (mod N)

    Parameter:
    - N: Die zu faktorisierende Zahl N.

    Rückgabe:
    - UV (Tupel): Ein StackUV Element mit einem Paar (u, v).
    """
    global Startq, Count1, StackUV
    UV = []
    while not StackUV:
        Count1 += 1
        StackUV.append((Startq^2, Startq^2 - Num))
        print('_', end='')
        Startq = Startq + 1
    UV = (StackUV.pop())
    return UV

In [21]:
def qr_trialdiv(v):
    """
    Die Funktion qr_trialdiv überprüft, ob eine gegebene Zahl v vollständig faktorisiert werden kann.

    Parameter:
    - v: die zu überprüfende Zahl

    Rückgabe:
    - true/false ob zutreffend

    """

    global Mat, Fbas, Flen, CurRow, Count2

    
    if v < 0:
        v = abs(v)
        Mat[CurRow,0] = 1
    
    for i in range(1, Flen):
        p = Fbas[i]

        
        while (v > 1) and (mod(v,p) == 0):
            v = int(v) // int(p)
            Mat[CurRow,i] += 1    
        
        #print(Mat[CurRow])
        #print(v)
    
        if v <= 1:
            Count2 += 1
            return True
    
    Mat[CurRow] = Mat[CurRow] + Mat[CurRow]
    return False


In [22]:
def findfactor(N, relvec):
    """
    Die Funktion findfactor sucht einen Faktor von N basierend auf den gegebenen Informationen .
    """

    global Uvec, Vvec, Fbas, Flen, Count1, Count2, nbfails

    x_squared = 1
    y_squared = 1


    for j in range(relvec.dimension()):
        
        for i in range(0, len(relvec[j+1])):
            if relvec[j+1][i] == 1: 
                x_squared = (x_squared * Uvec[i])
                y_squared = (y_squared * Vvec[i])
        x_squared = isqrt(x_squared) % N
        y_squared = isqrt(y_squared) % N
        d = gcd(x_squared + y_squared, N)   
        
    

        if d <= 1 or d == N:
            d = 0 # failed
        else: 
            if d > 1: # success
                print("Info about found data here ")
                print("Fbas = ", Fbas) 
                print("kernel = ", relvec[j+1])
                print("xi_sqaured = ", Uvec)
                print("xi_sqaured_minus_N = ", Vvec) 
                print("x = ", x_squared)
                print("y = ", y_squared)
            
                return d

    return d

In [23]:
N = 2041
QSfactorize(N)

quadratic sieve length 49 , factorbase 2 ... 7 of length 4
working _.__.______.__.______.___._.__._._.!Info about found data here 
Fbas =  [-1, 2, 3, 5, 7]
kernel =  (1, 0, 0, 0, 0, 0, 0, 0, 0, 1)
xi_sqaured =  [441, 529, 841, 961, 1369, 1600, 1681, 1849, 1936, 2025]
xi_sqaured_minus_N =  [-1600, -1512, -1200, -1080, -672, -441, -360, -192, -105, -16]
x =  945
y =  160

25 polynomials, 10 completely factorized quadratic residues


13

In [24]:
N = 156487973737
QSfactorize(N)

quadratic sieve length 9409 , factorbase 2 ... 103 of length 15
working ________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

294787

In [25]:
N = 28282936816460717 * 715471838565524927
QSfactorize(N)

quadratic sieve length 117050761 , factorbase 2 ... 10831 of length 717
working ________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [24]:
N = 2828293681646068747 * 715471838565524878741 * 155250434765112172389517
QSfactorize(N)

MemoryError: failed to allocate 410684877409 * 40 bytes